In [2]:
import scipy
import numpy as np
import pandas as pd
from pyDOE2 import ff2n, fullfact, ccdesign
import definitive_screening_design as dsd

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

pio.templates.default = "plotly_white"
pcolors = px.colors.qualitative.T10
pcolors25 = px.colors.qualitative.Alphabet

# DOE Introduction
<details>
<summary>
<font size="3" color="black">
<b>DOE Methods ⏏︎Click to open</b>
</font>
</summary>

- 相关的函数
1. **`fullfact`（全因素实验设计）：**
   - `fullfact` 函数来源于 `pyDOE2` 库，用于生成全因素（或全阵列）实验设计。在全因素实验设计中，每个因素（即变量）的每个水平都与其他因素的每个水平组合一次。这种设计方法能够评估所有可能的条件组合，从而帮助研究者完全理解不同因素之间的交互作用如何影响结果。
   - 适用场景：当你需要详尽无遗地研究多个因素对一个或多个响应变量的影响时，全因素实验设计非常有用。

2. **`ff2n`（2的n次方设计）：**
   - `ff2n` 函数也属于 `pyDOE2` 库，用于生成一个 2 的 n 次方设计。这种设计只考虑两个水平（通常是高和低）的每个因素，是一种分辨率较高的实验设计方法，用于快速评估主要效应，而不考虑交互作用。
   - 适用场景：在初步探索阶段，当需要快速检查哪些因素对结果有显著影响时，使用 2^n 设计可以节省时间和资源。

3. **`ccdesign`（中心复合设计）：**
   - `ccdesign` 函数用于创建中心复合设计（CCD），这是响应面方法（RSM）的一种形式，常用于建立一个因变量与一个或多个自变量之间的近似二次关系模型。中心复合设计包括立方点、星点（在每个轴上的高低水平外延）和中心点，这有助于评估曲线效应（非线性关系）。
   - 适用场景：当需要优化一个过程并且已知响应与因素之间的关系可能是非线性的，使用CCD可以更好地拟合和优化这种曲线关系。

4. **`definitive_screening_design`（DSD）：**
   - `definitive_screening_design` 或 `dsd` 是一种较新的实验设计方法，用于在单次实验中同时评估因素的主效应和二次交互作用。它需要的实验数量比全因素设计少，但提供的信息量多于传统的筛选设计。
   - 适用场景：当实验资源有限但需要评估多个因素及其交互作用时，DSD 提供了一种高效的解决方案。


- 下面五种实验设计方法都是用来高效地探索多个因素对结果的影响。下面，我将逐一介绍这些方法，并通过简单的数据示例来说明它们各自的特点和应用。

1. 拉丁超立方抽样 (Latin-Hypercube Sampling, LHS)

   拉丁超立方抽样是一种统计方法，用于生成近似随机的样本点。它确保每个参数的整个范围都被均匀探索。

   **示例**：
   假设我们有两个参数：温度（20°C 到 80°C）和时间（1小时到 5小时）。在拉丁超立方抽样中，这个范围会被等分，如果我们选择5个样本点，每个参数被分为5个区间，每个区间内随机选择一个值，但保证每个区间只被选择一次。

   | 样本编号 | 温度 (°C) | 时间 (小时) |
   |----------|-----------|------------|
   | 1        | 22        | 2.4        |
   | 2        | 56        | 4.1        |
   | 3        | 74        | 3.3        |
   | 4        | 39        | 1.2        |
   | 5        | 63        | 4.9        |

2. 2级全因子设计 (2-level Full-Factorial)

   这种设计考虑了所有可能的因素组合，每个因素有两个水平（通常是高和低）。

   **示例**：
   假设有两个因素：A（高和低）和 B（高和低）。

   | 样本编号 | 因素 A | 因素 B |
   |----------|--------|--------|
   | 1        | 低     | 低      |
   | 2        | 低     | 高      |
   | 3        | 高     | 低      |
   | 4        | 高     | 高      |

3. 3级全因子设计 (3-level Full-Factorial)

   这种设计每个因素有三个水平，通常是低、中、高，可以更细致地探索因素的影响。

   **示例**：
   同样假设有两个因素：A（低、中、高）和 B（低、中、高）。

   | 样本编号 | 因素 A | 因素 B |
   |----------|--------|--------|
   | 1        | 低     | 低     |
   | 2        | 低     | 中     |
   | 3        | 低     | 高     |
   | ...      | ...    | ...    |
   | 9        | 高     | 高     |

4. 中心复合设计 (Central Composite Design, CCD)

   CCD用于响应面方法，包括因子的两个水平、中心点和星形点，非常适合建模曲线关系。

   **示例**：
   因素 A 和 B，每个因素有低、中、高水平，以及星点。

   | 样本编号 | 因素 A | 因素 B |
   |----------|--------|--------|
   | 1        | 低     | 低     |
   | 2        | 高     | 高     |
   | 3        | 中     | 中     |  (中心点)
   | 4        | 最低   | 中     |  (星点)
   | 5        | 最高   | 中     |  (星点)

5. 确定性筛选设计 (Definitive Screening Design, DSD)

   DSD 是一种能同时评估因素主效应和交互效应的设计，用更少的实验次数比全因子设计更高效。

   **示例**：
   假设有三个因素：A、B 和 C，每个因素有三个水平。

   | 样本编号 | 因素 A | 因素 B | 因素 C |
   |----------|--------|--------|--------|
   | 1        | 低     |中     | 高     |
   | 2        | 中     | 高     | 低     |
   | 3        | 高     | 低     | 中     |
   | ...      | ...    | ...    | ...    |

以上是每种实验设计方法的简化示例，它们在实际应用中可以根据需要进行调整和扩展，以适应不同的研究和分析需求。
总的来说，这些实验设计方法在科学研究和工程应用中非常重要，尤其是在需要精确控制和优化多个变量时。使用这些方法可以帮助研究人员系统地探索、分析和优化过程，以便做出基于数据的决策。

</details>

In [3]:
def generate_doe(
    feed_start,
    feed_end,
    feed_rate,
    glc_0,
    vcd_0,
    num_runs,
    doe_design="Latin-Hypercube Sampling",
    filename=None,
    num_center_points=1,
    seed=123,
):
    var_lims = {
        "feed_start": feed_start,
        "feed_end": feed_end,
        "feed_rate": feed_rate,
        "glc_0": glc_0,
        "vcd_0": vcd_0,
    }

    rng = np.random.default_rng(seed)
    num_vars = len(var_lims)
    num_center_points = 1
    if type(num_runs) is tuple:
        num_runs = num_runs[0]
    num_samples = num_runs - num_center_points
    # determine which vars are part of DOE
    doe_var = [1 if type(v) is tuple and v[0] < v[1] else 0 for v in var_lims.values()]
    doe_var_idx = np.cumsum(doe_var) - 1

    if doe_design == "Latin-Hypercube Sampling":
        # sample points in the latin hypercube
        lhsampler = scipy.stats.qmc.LatinHypercube(
            d=sum(doe_var), scramble=True, seed=rng
        )
        doe_plan = lhsampler.random(n=num_samples)
    if doe_design == "2-level Full-Factorial":
        # 返回的设计矩阵原本包含-1和1，表示低高两个水平
        doe_plan = (ff2n(sum(doe_var)) + 1) / 2  # 将原始的 -1 和 1 转换为 0 和 1
    if doe_design == "3-level Full-Factorial":
        # 0, 1, 2, 表示低中高三个水平
        doe_plan = (
            fullfact([3] * sum(doe_var)) / 2
        )  # 除以2是为了将范围从原始的 {0, 1, 2} 调整到 {0, 0.5, 1}
    if doe_design == "Central-Composite":
        doe_plan = (
            ccdesign(sum(doe_var), center=(0, 1), face="cci") + 1
        ) / 2  # 将原始的 -1 和 1 转换为 0 和 1
    if doe_design == "Definitive-Screening":
        doe_plan = (
            dsd.generate(n_num=sum(doe_var), verbose=False).values + 1
        ) / 2  # 将原始的 -1 和 1 转换为 0 和 1
    if num_runs < len(doe_plan):
        print(
            f"\n The selected design requires {len(doe_plan)} runs, while only {num_runs} runs were selected!"
        )
        print(" Only a subset of created runs will be used!\n")
        subset_idx = np.sort(
            np.random.choice(range(len(doe_plan)), size=num_runs, replace=False)
        )
        doe_plan = doe_plan[subset_idx, :]

    # fill remaining unscaled vars
    doe_unscaled = np.ones([len(doe_plan) + 1, num_vars]) * 0.5
    for i in range(num_vars):
        if doe_var[i] == 1:
            doe_unscaled[num_center_points:, i] = doe_plan[:, doe_var_idx[i]]

    # scale all vars according to var_lims
    doe_scaled = doe_unscaled
    for i, k in enumerate(var_lims.keys()):
        if doe_var[i] == 1:
            doe_scaled[:, i] = (
                doe_unscaled[:, i] * (var_lims[k][1] - var_lims[k][0]) + var_lims[k][0]
            )
        else:
            doe_scaled[:, i] = var_lims[k][0]

    doe_names = ["feed_start", "feed_end", "feed_rate", "glc_0", "vcd_0"]
    doe_df = pd.DataFrame(doe_scaled, columns=doe_names)
    doe_df.to_csv(filename, index=False)

    return doe_scaled

In [4]:
def plot_doe(
    doe_scaled,
    feed_start=(1., 4.),
    feed_end=(8., 12.),
    feed_rate=(5., 20.),
    glc_0=(10., 80.0),
    vcd_0=(0.1, 1.0),
    doe_design="Latin-Hypercube Sampling",
):
    fig = go.Figure(
        data=go.Parcoords(
            line_color="blue",
            dimensions=list(
                [
                    dict(
                        label="feed_start",
                        values=doe_scaled[:, 0].round(),
                        range=list(feed_start),
                    ),
                    dict(
                        label="feed_end",
                        values=doe_scaled[:, 1].round(),
                        range=list(feed_end),
                    ),
                    dict(
                        label="feed_rate",
                        values=doe_scaled[:, 2],
                        range=list(feed_rate),
                    ),
                    dict(label="glc_0", values=doe_scaled[:, 3], range=list(glc_0)),
                    dict(label="vcd_0", values=doe_scaled[:, 4], range=list(vcd_0)),
                ]
            ),
        )
    )
    fig.update_layout(
        showlegend=False,
        title_text=f"Generated design of experiments using {doe_design} with {len(doe_scaled)} runs",
        # width=1600,
    )

    fig.show();

# Setting
For each of the manipulated variables defined in the section above, the use can define the limits of the exploration space (first value: lower limit; second value: upper limit).

- 用户可以为每个操作变量定义探索空间的限制（最小值和最大值）。这些变量包括：
  - `feed_start`：喂料开始时间。
  - `feed_end`：喂料结束时间。
  - `feed_rate`：喂料速率。
  - `glc_0`：初始葡萄糖浓度。
  - `vcd_0`：初始可行细胞密度。

- 指定DOE_DESIGN方法, 选项如下， 默认是"Latin-Hypercube Sampling"：
  - "Latin-Hypercube Sampling",
  - "2-level Full-Factorial",
  - "3-level Full-Factorial",
  - "Central-Composite",
  - "Definitive-Screening",

- 指定要生成的数据数量：NUM_RUNS

In [5]:
FEED_START = (1., 4.)
FEED_END = (8., 12.)
GLC_FEED_RATE = (5., 20.)
GLC_0 = (10., 80.0)
VCD_0 = (0.1, 1.0)

NUM_RUNS = 50
DOE_DESIGN = "Latin-Hypercube Sampling"

FILENAME = 'dataset/datahow_concise/owu_doe.csv'

# DOE Data Generation

In [6]:
doe_scaled = generate_doe(
    feed_start=FEED_START,
    feed_end=FEED_END,
    feed_rate=GLC_FEED_RATE,
    glc_0=GLC_0,
    vcd_0=VCD_0,
    num_runs=NUM_RUNS,
    doe_design=DOE_DESIGN,
	filename=FILENAME,
	seed=42
)
doe_scaled[:5]

array([[ 2.5       , 10.        , 12.5       , 45.        ,  0.55      ],
       [ 1.61734219, 11.92563374, 17.89492066, 65.27240227,  0.92736346],
       [ 1.29540237,  8.81660178,  9.05549996, 18.35659831,  0.99295902],
       [ 3.65841035,  9.61832993,  5.84203168, 60.10300546,  0.58764499],
       [ 2.09711237,  8.49559394, 18.77104452, 14.64990119,  0.56821711]])

# Visualization for DOE

使用拉丁超立方抽样（LHD）来均匀地映射变量的五维空间。这是一种常用于工程和科学研究中的实验设计方法，特别适合于需要在多个维度上探索变量的情况。这里的详细解释如下：

- `plot_doe`函数负责根据输入的参数生成实验设计，并使用`go.Parcoords`（平行坐标图）来可视化设计。平行坐标图是一种非常有用的多维数据可视化工具，能够展示每个实验运行中各变量的值。
- 平行坐标图解释
	- 图中的每根线代表一个实验运行的配置。
	- 线通过的位置表示该运行中各变量的值。
	- 用户可以通过滑块调整变量的范围，这影响到显示的实验配置。

In [7]:
plot_doe(
	doe_scaled,
    feed_start=FEED_START,
    feed_end=FEED_END,
    feed_rate=GLC_FEED_RATE,
    glc_0=GLC_0,
    vcd_0=VCD_0,
    doe_design=DOE_DESIGN,
)